In [14]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import torch
import torch.nn as nn
import re
from collections import Counter
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
import logging

logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "bert-base-multilingual-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\harish-4072\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\harish-4072\.cache\huggingface\hub\models--bert-base-multilingual-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [15]:
train_df = pd.read_csv(r"D:\elggak\kaggle\contradictory-my-dear-watson\train.csv")
test_df = pd.read_csv(r"D:\elggak\kaggle\contradictory-my-dear-watson\test.csv")

In [16]:

id = train_df['id']
test_id = test_df['id']

In [20]:
train_df['premise_tokens'] = train_df['premise'].apply(lambda x: len(tokenizer(x)['input_ids']))
train_df['hypothesis_tokens'] = train_df['hypothesis'].apply(lambda x: len(tokenizer(x)['input_ids']))
train_df['combined_tokens'] = train_df.apply(lambda x: len(tokenizer(x['premise'], x['hypothesis'])['input_ids']), axis=1)

# Sort by combined token length and display lengths
length_analysis = train_df[['premise_tokens', 'hypothesis_tokens', 'combined_tokens']].sort_values('hypothesis_tokens', ascending=False)

# Display top 10 longest examples
print("Top 10 longest examples (by tokens):")
print(length_analysis.head(100))

# Get some statistics
print("\nToken length statistics:")
print(length_analysis.describe())

# Optional: Display the actual text of the longest example
longest_idx = length_analysis.index[0]
print("\nLongest example:")
print(f"Premise: {train_df.loc[longest_idx, 'premise']}")
print(f"Hypothesis: {train_df.loc[longest_idx, 'hypothesis']}")
print(f"Total tokens: {length_analysis.loc[longest_idx, 'combined_tokens']}")

Top 10 longest examples (by tokens):
       premise_tokens  hypothesis_tokens  combined_tokens
6809               44                 66              109
3699               55                 64              118
8862               49                 62              110
333                38                 58               95
2159               54                 55              108
...               ...                ...              ...
10526              50                 39               88
1850               18                 39               56
2760               32                 39               70
10278              73                 39              111
427                54                 38               91

[100 rows x 3 columns]

Token length statistics:
       premise_tokens  hypothesis_tokens  combined_tokens
count    12120.000000       12120.000000     12120.000000
mean        29.881683          15.894719        44.776403
std         17.779207           6.646482   

In [4]:
class ElementaryDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.premise = df["premise"]
        self.hypothesis = df["hypothesis"]
        if 'label' in df.columns:
            self.labels = df['label']
        else:
            self.labels = None
        self.tokenizer = tokenizer
        self.maxlength = max_length

    def __len__(self):
        return len(self.premise)

    def __getitem__(self,idx):
        encoded = self.tokenizer(self.premise[idx], self.hypothesis[idx], padding="max_length", max_length=self.maxlength, truncation=True, return_tensors="pt")
        if self.labels is not None:
            return {
                "input_ids": encoded["input_ids"].squeeze(0),
                "token_ids": encoded["token_type_ids"].squeeze(0),
                "attention_mask": encoded["attention_mask"].squeeze(0),
                "labels": torch.tensor(self.labels[idx], dtype=torch.long)  
            }
        return {
                "input_ids": encoded["input_ids"].squeeze(0),
                "token_ids": encoded["token_type_ids"].squeeze(0),
                "attention_mask": encoded["attention_mask"].squeeze(0)
            }
        

In [5]:
data = ElementaryDataset(train_df, tokenizer, max_length=64)

In [6]:
train_size = int(0.8 * len(data))
val_size = len(data) - train_size
train_dataset, val_dataset = random_split(data, [train_size, val_size])
test_dataset = ElementaryDataset(test_df, tokenizer, max_length=64)

In [7]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=torch.cuda.is_available()
)

# Train model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
trainer.train()

C:\Users\harish-4072\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been

Epoch,Training Loss,Validation Loss
1,No log,0.849590


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

TrainOutput(global_step=303, training_loss=0.9644183168316832, metrics={'train_runtime': 3983.4282, 'train_samples_per_second': 2.434, 'train_steps_per_second': 0.076, 'total_flos': 318893462286336.0, 'train_loss': 0.9644183168316832, 'epoch': 1.0})

In [9]:
predictions = trainer.predict(test_dataset)
logits = predictions.predictions.to(device)
probs = torch.softmax(torch.tensor(logits), dim=-1)  # Convert logits to probabilities
pred_labels = torch.argmax(probs, dim=-1)


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [12]:
df = pd.DataFrame({"id": test_id, "target": pred_labels.cpu().tolist()})
df.to_csv(r"C:\\users\\harish-4072\\downloads\\elementary_predictions.csv", index=False)